In [11]:
import os

os.environ["HF_HOME"] = "/teamspace/studios/this_studio/weights"
os.environ["TORCH_HOME"] = "/teamspace/studios/this_studio/weights"

import gc
import re
import uuid
import textwrap
import subprocess
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext

from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

from rag_101.retriever import (
    load_embedding_model,
    load_reranker_model
)

In [12]:
# allows nested access to the event loop
nest_asyncio.apply()

In [13]:
# setting up the llm
llm=Ollama(model="llama3", request_timeout=60.0)

# setting up the embedding model
lc_embedding_model = load_embedding_model()
embed_model = LangchainEmbedding(lc_embedding_model)

In [14]:
# utility functions
def parse_github_url(url):
    pattern = r"https://github\.com/([^/]+)/([^/]+)"
    match = re.match(pattern, url)
    return match.groups() if match else (None, None)

def clone_github_repo(repo_url):    
    try:
        print('Cloning the repo ...')
        result = subprocess.run(["git", "clone", repo_url], check=True, text=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        print(f"Failed to clone repository: {e}")
        return None


def validate_owner_repo(owner, repo):
    return bool(owner) and bool(repo)

In [15]:
# Setup a query engine

def setup_query_engine(github_url):
    
    owner, repo = parse_github_url(github_url)
    
    if validate_owner_repo(owner, repo):
        # Clone the GitHub repo & save it in a directory
        input_dir_path = f"/teamspace/studios/this_studio/{repo}"

        if os.path.exists(input_dir_path):
            pass
        else:
            clone_github_repo(github_url)
        
        loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".py", ".ipynb", ".js", ".ts", ".md"],
            recursive=True
        )

        try:
            docs = loader.load_data()

            # ====== Create vector store and upload data ======
            Settings.embed_model = embed_model
            index = VectorStoreIndex.from_documents(docs, show_progress=True)
            # ====== Setup a query engine ======
            Settings.llm = llm
            query_engine = index.as_query_engine(similarity_top_k=4)
            
            # ====== Customise prompt template ======
            qa_prompt_tmpl_str = (
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "You are llama3, a large language model developed by Meta AI. Surya has integrated you into this environment so you can answer any user's coding questions! Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
            "Query: {query_str}\n"
            "Answer: "
            )
            qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

            query_engine.update_prompts(
                {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
            )

            if docs:
                print("Data loaded successfully!!")
                print("Ready to chat!!")
            else:
                print("No data found, check if the repository is not empty!")
            
            return query_engine

        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print('Invalid github repo, try again!')
        return None

In [16]:
# Provide url to the repository you want to chat with
github_url = "https://github.com/Lightning-AI/lit-gpt"

query_engine = setup_query_engine(github_url=github_url)

Cloning the repo ...


Parsing nodes:   0%|          | 0/325 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/535 [00:00<?, ?it/s]

Data loaded successfully!!
Ready to chat!!


In [17]:
response = query_engine.query('Tell me in detail about how to fine tune using lit-gpt?')
display(Markdown(str(response)))

Hello there! I'm llama3, and I'm here to help you with your query.

To finetune a model using LitGPT, you can follow these steps:

**Step 1: Choose the Finetuning Method**
LitGPT provides several methods for supervised instruction finetuning. The supported methods are:
```bash
litgpt finetune full
litgpt finetune lora
litgpt finetune adapter
litgpt finetune adapter_v2
```
In this example, I'll use the LoRA (Low-Rank Adaptation) method.

**Step 2: Prepare Your Dataset**
Your dataset should be formatted in a specific way. LitGPT supports datasets with an `input` field and those without. For instruction-finestuning, you can use the following format:
```
{
    "input": "<instruction_text>",
    "target": "<target_text>"
}
```
Alternatively, your dataset might contain an `input` field.

**Step 3: Download a Pretrained Model**
To start finetuning, you need to download a pretrained model. For example:
```bash
litgpt download --repo_id microsoft/phi-2
```
This will download the `microsoft/phi-2` model.

**Step 4: Configure Finetuning with LitGPT**
You can use command-line arguments and configuration files to finetune your model. Here's an example using LoRA:
```bash
litgpt finetune lora \
    --config config_hub/finetune/phi-2/lora.yaml \
    --train.max_steps 5
```
This will finetune the downloaded `microsoft/phi-2` model using LoRA and train for a maximum of 5 steps.

**Step 5: Run Finetuning**
Once you've configured LitGPT, you can run the finetuning process:
```bash
litgpt finetune lora ...
```
This will start training your model on the prepared dataset.

That's it! You should now have a good understanding of how to fine-tune using LitGPT.